In [1]:
import sqlite3

In [2]:
def query(db, q, p=()):
    with sqlite3.connect(db) as con:
        cur = con.cursor()
        r = cur.execute(q, p)
    return r.fetchall()

In [8]:
bigram = "C:/Users/larsj/Documents/ngram/bigram-one-row.db"
unigram = "C:/Users/larsj/Documents/ngram/unigram-one-row.db"

In [7]:
query(bigram, "select * from sqlite_master")

[('table',
  'bigram',
  'bigram',
  2,
  'CREATE TABLE bigram (freq int, lang varchar, first varchar, second varchar, json text, pmi float)'),
 ('index',
  '_lfsf_',
  'bigram',
  5857784,
  'CREATE INDEX _lfsf_ on bigram (lang,first,second,freq)'),
 ('index',
  '_lf_',
  'bigram',
  6834551,
  'CREATE INDEX _lf_ on bigram(lang,freq)'),
 ('table',
  'sqlite_stat1',
  'sqlite_stat1',
  7127480,
  'CREATE TABLE sqlite_stat1(tbl,idx,stat)'),
 ('index',
  '_fsf_',
  'bigram',
  7127481,
  'CREATE INDEX _fsf_ on bigram (first,second,freq)'),
 ('index',
  '_sf_',
  'bigram',
  7600914,
  'CREATE INDEX _sf_ on bigram(second,freq)'),
 ('index', '_f_', 'bigram', 7943929, 'CREATE INDEX _f_ on bigram(freq)'),
 ('index',
  '_ff_',
  'bigram',
  6411567,
  'CREATE INDEX _ff_ on bigram(first,freq)')]

In [6]:
query(bigram, "ALTER TABLE bigram ADD pmi float")

[]

In [9]:
query(unigram, "select * from sqlite_master")

[('table',
  'unigram',
  'unigram',
  2,
  'CREATE TABLE unigram (freq int, lang varchar, first varchar, json text)'),
 ('index',
  '_lff_',
  'unigram',
  1393414,
  'CREATE INDEX _lff_ on unigram(lang,first,freq)'),
 ('index',
  '_lf_',
  'unigram',
  1511081,
  'CREATE INDEX _lf_ on unigram(lang,freq)'),
 ('table',
  'sqlite_stat1',
  'sqlite_stat1',
  1576805,
  'CREATE TABLE sqlite_stat1(tbl,idx,stat)'),
 ('index',
  '_ff_',
  'unigram',
  1576806,
  'CREATE INDEX _ff_ on unigram(first,freq)'),
 ('index', '_f_', 'unigram', 1675757, 'CREATE INDEX _f_ on unigram(freq)')]

In [14]:
query(bigram, "select sum(freq) from bigram where lang='nob'")

[(8639060175,)]

In [20]:
query(bigram, "select first, second, freq from bigram where lang='nob' and first = 'spiser' order by freq desc limit 20")

[('spiser', ',', 16389),
 ('spiser', '.', 14512),
 ('spiser', 'og', 8983),
 ('spiser', 'vi', 6160),
 ('spiser', 'ikke', 5160),
 ('spiser', 'middag', 5146),
 ('spiser', 'de', 4955),
 ('spiser', 'den', 4320),
 ('spiser', 'opp', 3758),
 ('spiser', 'av', 3640),
 ('spiser', 'en', 3439),
 ('spiser', 'for', 3432),
 ('spiser', 'frokost', 3401),
 ('spiser', 'i', 3326),
 ('spiser', 'det', 2951),
 ('spiser', 'jeg', 2747),
 ('spiser', 'du', 2712),
 ('spiser', 'på', 2565),
 ('spiser', 'med', 2277),
 ('spiser', 'sammen', 2197)]

In [43]:
def check(w):
    with sqlite3.connect(bigram) as con:
        cur = con.cursor()
        cur.execute("attach '{db}' as unigram".format(db=unigram))
        cur.execute("""
        select b.first, b.second, b.freq, uf.freq, us.freq, b.freq*b.freq*1.0/(uf.freq*us.freq) as pmi from bigram as b, unigram as uf, unigram as us
        where b.first = uf.first and b.second = us.first
        and b.first = ? and b.lang='nob' and uf.lang='nob' and us.lang='nob'
        order by pmi desc limit 20 
        """, (w,))
        r =  cur.fetchall()
    return r

In [49]:
check(',')

[(',', 'men', 21397468, 471880905, 24560995, 0.03950448557038251),
 (',', 'og', 46017522, 471880905, 259886630, 0.017267525896340064),
 (',', 'som', 17268688, 471880905, 128038992, 0.004935646371869148),
 (',', 'mens', 2779849, 471880905, 4590100, 0.0035676961601516124),
 (',', 'sa', 3563564, 471880905, 11070715, 0.0024308661808788296),
 (',', 'f', 3151424, 471880905, 10815503, 0.0019459628744578654),
 (',', 'så', 5011879, 471880905, 28430919, 0.0018723104151495233),
 (',', 'der', 3749128, 471880905, 17035133, 0.0017485683122722005),
 (',', 'hvor', 2529001, 471880905, 9287831, 0.0014593225521253395),
 (',', 'at', 7387731, 471880905, 93020220, 0.0012434043469073848),
 (',', 'da', 3052929, 471880905, 16500805, 0.0011970047476890996),
 (',', 'dvs.', 697377, 471880905, 892663, 0.001154556796380065),
 (',', '»', 4751195, 471880905, 46888112, 0.0010202592972851636),
 (',', 'slik', 2104889, 471880905, 9555137, 0.0009826278640522154),
 (',', 'idet', 705447, 471880905, 1122914, 0.00093918226220

In [72]:

with sqlite3.connect(bigram) as con:
    cur = con.cursor()
    cur.execute("attach '{db}' as unigram".format(db=unigram))
    cur.execute("""
    update bigram set pmi = (select bigram.freq*1.0/(uf.freq*us.freq) from unigram as uf, unigram as us
    where bigram.first = uf.first and bigram.second = us.first
    and bigram.lang = uf.lang and bigram.lang = us.lang)
    """)
    

In [101]:
query(bigram, "select first, second, freq, pmi from bigram where second='gutter' and lang = 'nob' order by pmi desc limit 20")

[('slikk-vorne', 'gutter', 8, 2.490373151287056e-06),
 ('Kjække', 'gutter', 33, 3.475755760959848e-07),
 ('halvvoksne', 'gutter', 573, 3.3733069818205894e-07),
 ('Badende', 'gutter', 194, 3.233336943892921e-07),
 ('Wets', 'gutter', 13, 3.223159941378159e-07),
 ('Kjekke', 'gutter', 107, 2.9947918889728206e-07),
 ('Tøffe', 'gutter', 109, 1.98042806534744e-07),
 ('halwoksne', 'gutter', 51, 1.9708297869668253e-07),
 ('kjekke', 'gutter', 1459, 1.6152834233754018e-07),
 ('halvstore', 'gutter', 60, 1.1443171389508255e-07),
 ('HKR', 'gutter', 9, 1.0208513423798154e-07),
 ('uskikkelige', 'gutter', 142, 9.422953835104387e-08),
 ('Problematiske', 'gutter', 21, 9.150087978095909e-08),
 ('kjække', 'gutter', 176, 8.905434060950642e-08),
 ('LR', 'gutter', 769, 8.890703059906798e-08),
 ('Flinke', 'gutter', 66, 8.02910145388901e-08),
 ('fylkesrenn', 'gutter', 14, 7.705968002509063e-08),
 ('Kretsmester', 'gutter', 22, 6.126912076973622e-08),
 ('ukonfirmerte', 'gutter', 20, 6.057664422049596e-08),
 ('DR'